In [1]:
!nvidia-smi

Fri Jun 24 06:00:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%pip install -q transformers[sentencepiece] datasets

In [1]:
import sys
sys.path.append("/content/gdrive/MyDrive/Kaggle/NLP/PatentPhraseMatching/src")

### In this notebook we experiment by finetuning a bert-for-patents model on competition data by adding patent section as special token to the tokenizer vocab.

In [2]:
import colab_utils
import numpy as np
import pandas as pd
import torch
import os
from sklearn.model_selection import StratifiedKFold
from datasets import Dataset
from torch.utils.data import DataLoader
import transformers
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import torch.multiprocessing as mp
from transformers import logging
import warnings

logging.set_verbosity_error()
logging.set_verbosity_warning()
warnings.filterwarnings('ignore')


In [3]:
class Config:
    MODEL_NAME = "deberta-v3-small"
    DATA_PATH = "/content/gdrive/MyDrive/Kaggle/NLP/PatentPhraseMatching/data/"
    VAL_PREDS_PATH = "/content/gdrive/MyDrive/Kaggle/NLP/PatentPhraseMatching/preds/"
    # location where trained model weights are saved
    OUT_DIR = "/content/gdrive/MyDrive/Kaggle/NLP/PatentPhraseMatching/model/deberta-v3-small/"
    RUNTIME = "COLAB"
    RANDOM_STATE = 42
    BATCH_SIZE = 32
    EVAL_BATCH_SIZE = 64
    NUM_LABELS = 1
    LABEL_COL = "score"
    NUM_FOLDS = 5
    RUN_ALL_FOLDS = False
    NUM_EPOCHS = 2
    NUM_WORKERS = mp.cpu_count()
    TRANSFORMER_CHECKPOINT = "microsoft/deberta-v3-small"
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    SUBSET_ROWS_FRAC = 0.05
    TRAIN_ON_SUBSET = False
    RANDOM_SEED = 42

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [4]:
training_args_dict = dict(
    evaluation_strategy="epoch",
    save_strategy='epoch',        
    num_train_epochs=Config.NUM_EPOCHS,
    per_device_train_batch_size=Config.BATCH_SIZE,
    per_device_eval_batch_size=Config.EVAL_BATCH_SIZE,
    weight_decay = 0.01,
    learning_rate = 4e-5,
    warmup_ratio = 0.1,
    gradient_accumulation_steps = 4,
    fp16 = True,
    lr_scheduler_type = "linear",
    # Number of checkpoints to save for each model
    save_total_limit = 1,
    #  Whether or not to load the best model found during training at the end of training.
    load_best_model_at_end=True,
    # Use in conjunction with `load_best_model_at_end` to specify the metric to use to compare two different
    # models. Must be the name of a metric returned by the evaluation with or without the prefix `"eval_"`. Will
    # default to `"loss"` if unspecified and `load_best_model_at_end=True` (to use the evaluation loss).
    # If you set this value, `greater_is_better` will default to `True`. Don't forget to set it to `False` if
    # your metric is better when lower.
    metric_for_best_model="pearson",
    greater_is_better=True,
    adam_epsilon=1e-6,
    #warmup_steps=1000
    log_level="warning",
    group_by_length=True    
)    

In [5]:
colab_utils.empty_gdrive_trash()

The below files were cleared from trash
[]


In [6]:
df_train = pd.read_csv(Config.DATA_PATH + "train.csv")
df_train["section"] = df_train.context.str[0]
df_test = pd.read_csv(Config.DATA_PATH + "test.csv")
df_titles = pd.read_csv(Config.DATA_PATH + "titles.csv")

In [7]:
from sklearn.preprocessing import LabelEncoder

anchor_encoder = LabelEncoder()
df_train["anchor_map"] = anchor_encoder.fit_transform(df_train["anchor"])
df_train["context_map"] = anchor_encoder.fit_transform(df_train["context"])
df_train["anchor_context_map"] = df_train["anchor_map"].astype(str).str.cat(df_train["context_map"].astype(str), sep="_")
# Score is not really a continuous value here as there are just five distinct values. But since it is float it needs to be converted
# to categorical value before we can perform stratified split on score
df_train["score_map"] = df_train["score"].map({0.00: 0, 0.25: 1, 0.50: 2, 0.75: 3, 1.00: 4})

In [8]:
from sklearn import model_selection

def strat_group_kfold_dataframe(df, target_col_name, group_col_name, num_folds=Config.NUM_FOLDS):
    # we create a new column called kfold and fill it with -1
    df["kfold"] = -1
    # randomize of shuffle the rows of dataframe before splitting is done
    df = df.sample(frac=1, random_state=Config.RANDOM_SEED).reset_index(drop=True)
    # get the target data
    y = df[target_col_name].values    
    groups = df[group_col_name].values
    # stratify data using anchor as group and score as target
    skf = model_selection.StratifiedGroupKFold(n_splits=num_folds, shuffle=True, random_state=Config.RANDOM_SEED)
    for fold, (train_index, val_index) in enumerate(skf.split(X=df, y=y, groups=groups)):
        df.loc[val_index, "kfold"] = fold        
    return df     

In [9]:
def strat_kfold_dataframe(df, target_col_name, num_folds=5):
    # we create a new column called kfold and fill it with -1
    df["kfold"] = -1
    # randomize of shuffle the rows of dataframe before splitting is done
    df.sample(frac=1, random_state=Config.RANDOM_STATE).reset_index(drop=True)
    y = df[target_col_name].values
    skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=Config.RANDOM_STATE)
    # stratification is done on the basis of y labels, a placeholder for X is sufficient
    for fold, (train_idx, val_idx) in enumerate(skf.split(X=df, y=y)):
        df.loc[val_idx, "kfold"] = fold
    return df

In [10]:
if Config.TRAIN_ON_SUBSET:
    print(f"Selecting {Config.SUBSET_ROWS_FRAC * 100}% training data")
    df_train = df_train.sample(frac=Config.SUBSET_ROWS_FRAC, random_state=Config.RANDOM_SEED).reset_index(drop=True)

# Since the target column (score) is continuous, we need to create bins out of the target column
# df_train.loc[:, "bins"] = pd.cut(df_train.score, bins=5, labels=[0,1,2,3,4])
# df_train = strat_kfold_dataframe(df_train, target_col_name="bins", num_folds=Config.NUM_FOLDS)

# Now do a stratified group k fold on the bins column (which is a categorical column) and anchor as groups
df_train = strat_group_kfold_dataframe(df_train, target_col_name="score_map", group_col_name="anchor_context_map", num_folds=Config.NUM_FOLDS)            
# drop the bin column
# df_train = df_train.drop(["bins"], axis=1)
# df_train = df_train.drop(["anchor_map", "score_map"], axis=1)

In [11]:
# Let us check if the stratification has been done correctly
# The mean of score column should be similar across folds 
fold_score_mean = []
fold_anchor_context_maps = []
for fold in range(Config.NUM_FOLDS):
    df_train_fold = df_train[df_train.kfold == fold]
    fold_score_mean.append(np.mean(df_train_fold.score.values))
    fold_anchor_context_maps.append(set(df_train_fold.anchor_context_map.unique()))
fold_score_mean

[0.3557356434260165,
 0.35813229056203605,
 0.36268028846153844,
 0.36315899290582837,
 0.37085976039464413]

In [12]:
# check each of the folds has no common anchor value
def check_disjoint(start, fold_anchor_context_maps):
    for i in range(start, 4):
        for j in range(i+1, 5):
            if fold_anchor_context_maps[i].isdisjoint(fold_anchor_context_maps[j]):
                print(f"anchor context map for fold {i} and {j} are disjoint")

check_disjoint(0, fold_anchor_context_maps)                

anchor context map for fold 0 and 1 are disjoint
anchor context map for fold 0 and 2 are disjoint
anchor context map for fold 0 and 3 are disjoint
anchor context map for fold 0 and 4 are disjoint
anchor context map for fold 1 and 2 are disjoint
anchor context map for fold 1 and 3 are disjoint
anchor context map for fold 1 and 4 are disjoint
anchor context map for fold 2 and 3 are disjoint
anchor context map for fold 2 and 4 are disjoint
anchor context map for fold 3 and 4 are disjoint


In [13]:
df_titles.head()

,code,title,section,class,subclass,group,main_group
0,A,HUMAN NECESSITIES,A,NaN,NaN,NaN,NaN
1,A01,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTI...,A,1.0,NaN,NaN,NaN
2,A01B,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS...,A,1.0,B,NaN,NaN
3,A01B1/00,Hand tools (edge trimmers for lawns A01G3/06 ...,A,1.0,B,1.0,0.0
4,A01B1/02,Spades; Shovels {(hand-operated dredgers E02F3...,A,1.0,B,1.0,2.0


In [14]:
df_train = pd.merge(
    left = df_train,
    right = df_titles[["code", "title"]],
    how = "inner",
    left_on = "context",
    right_on = "code"
)

In [15]:
df_train.anchor_context_map.value_counts()

555_89     84
129_47     79
324_49     79
430_104    74
129_48     73
           ..
525_62      1
649_62      1
480_11      1
485_11      1
727_101     1
Name: anchor_context_map, Length: 1699, dtype: int64

In [16]:
# For each anchor, context group (i.e. set of records having same anchor and context values), concatenate the target phrases
# key is unique anchor_context_map , value is concatenation of target phrases of all records for that unique anchor_context_map
anc_ctx_targets = {}
for anchor_context_map in df_train.anchor_context_map.unique():
    df_train_sub = df_train[df_train.anchor_context_map == anchor_context_map]
    anchor_context_target_text = ",".join(df_train_sub.target)    
    anc_ctx_targets[anchor_context_map] = anchor_context_target_text

df_train["anchor_context_targets"] = df_train.anchor_context_map.map(anc_ctx_targets)
df_train["anc_ctx_tgt_len"] = df_train["anchor_context_targets"].apply(lambda text: len(text.split()))
df_train = df_train.sort_values(by=["anc_ctx_tgt_len"], ascending=False)
# df_train = df_train.head(500)
# df_train = df_train[df_train.anchor_context_map == "555_89"]

In [17]:
from transformers import AutoTokenizer, DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained(Config.TRANSFORMER_CHECKPOINT)
# DataCollatorWithPadding pads each batch to the longest sequence length
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [18]:
df_train['sectok'] = '[' + df_train.section + ']'
sectoks = list(df_train.sectok.unique())
print(f"Additional special tokens: {sectoks}")
tokenizer.add_special_tokens({'additional_special_tokens': sectoks})

Additional special tokens: ['[G]', '[C]', '[H]', '[B]', '[A]', '[E]', '[F]', '[D]']


8

In [19]:
sep = " " + tokenizer.sep_token + " "
df_train["inputs"] = df_train.sectok + sep + df_train.anchor + sep + df_train.target + sep + df_train.title + sep + df_train.anchor_context_targets
df_train["inputs"] = df_train["inputs"].apply(lambda x: x.lower())
df_train.head()

,id,anchor,target,context,score,section,anchor_map,context_map,anchor_context_map,score_map,kfold,code,title,anchor_context_targets,anc_ctx_tgt_len,sectok,inputs
18583,426b5d4ee52dfbba,reflection type liquid crystal display,reflective mode liquid display,G02,0.50,G,555,89,555_89,2,4,G02,OPTICS,"liquid matrix type crystal,lcd displays reflec...",213,[G],[g] [sep] reflection type liquid crystal displ...
19157,d583a6c02fed7b2a,reflection type liquid crystal display,reflection mode lcd crystal,G02,0.50,G,555,89,555_89,2,4,G02,OPTICS,"liquid matrix type crystal,lcd displays reflec...",213,[G],[g] [sep] reflection type liquid crystal displ...
19227,668eb746e5b96f9e,reflection type liquid crystal display,mobile,G02,0.25,G,555,89,555_89,1,4,G02,OPTICS,"liquid matrix type crystal,lcd displays reflec...",213,[G],[g] [sep] reflection type liquid crystal displ...
19221,b0e707f934a27619,reflection type liquid crystal display,reflection type crystal display,G02,0.50,G,555,89,555_89,2,4,G02,OPTICS,"liquid matrix type crystal,lcd displays reflec...",213,[G],[g] [sep] reflection type liquid crystal displ...
19217,722fbf83a2054afa,reflection type liquid crystal display,liquid crystal device,G02,0.50,G,555,89,555_89,2,4,G02,OPTICS,"liquid matrix type crystal,lcd displays reflec...",213,[G],[g] [sep] reflection type liquid crystal displ...


In [20]:
def tokenize_text(tokenizer, with_labels, row):
    encoding = tokenizer(
        text = row["inputs"],
        padding = False,
        truncation = True,
        # maximum possible sequence length (for inputs column). Sequences exceeding this length will be truncated
        max_length = 512
    )
    if with_labels:
        encoding["labels"] = row[Config.LABEL_COL]
    return encoding

In [21]:
from functools import partial

preprocess_train_data = partial(tokenize_text, tokenizer, True)  
preprocess_test_data = partial(tokenize_text, tokenizer, False)  

In [22]:
def get_fold_dls(fold, df):
    train_df = df[df.kfold != fold].reset_index(drop=True)
    valid_df = df[df.kfold == fold].reset_index(drop=True)
    ds_train_raw = Dataset.from_pandas(train_df)
    ds_valid_raw = Dataset.from_pandas(valid_df)
    raw_ds_col_names = ds_train_raw.column_names    
    ds_train = ds_train_raw.map(preprocess_train_data, batched=True, batch_size=1000, remove_columns=raw_ds_col_names)
    ds_valid = ds_valid_raw.map(preprocess_train_data, batched=True, batch_size=1000, remove_columns=raw_ds_col_names)    
    return train_df, valid_df, ds_train, ds_valid

In [23]:
# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred        
#     if Config.NUM_LABELS == 1:
#         y_preds = predictions.reshape(len(predictions))
#     else:
#         y_preds = np.argmax(predictions, axis=1)
#     return {
#         'eval_pearson': np.corrcoef(y_preds, labels)[0][1]
#     }

In [24]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.reshape(len(predictions))
    return {
        'pearson': np.corrcoef(predictions, labels)[0][1]
    }

In [25]:
# def get_oof_preds(trainer, ds_val, df_val_fold):
#     oof_outputs = trainer.predict(ds_val)
#     if Config.NUM_LABELS == 1:
#         y_preds_proba = oof_outputs.predictions
#         oof_predictions = np.argmax(y_preds_proba, axis=1)
#     else:
#         oof_predictions = oof_outputs.predictions.reshape(-1)
#     df_val_fold["val_preds"] = oof_predictions
#     return df_val_fold

In [26]:
def get_oof_preds(trainer, ds_val, df_val_fold):
    oof_outputs = trainer.predict(ds_val)
    oof_predictions = oof_outputs.predictions.reshape(-1)
    df_val_fold["val_preds"] = oof_predictions
    return df_val_fold

In [28]:
import gc
import custom_transformer_heads
import utils

df_val_preds = pd.DataFrame()
tok_vocab = tokenizer.get_vocab()
for fold in range(Config.NUM_FOLDS):
    fold_str = f"fold{fold}"
    print(f"Running training for {fold_str}")
    df_train_fold, df_val_fold, ds_train, ds_val = get_fold_dls(fold, df_train)
    training_args_dict["output_dir"] = Config.OUT_DIR + fold_str
    training_args = TrainingArguments(**training_args_dict, report_to=None)
    model = custom_transformer_heads.DebertaV2ForSeqClfMeanPooling.from_pretrained(Config.TRANSFORMER_CHECKPOINT, num_labels=Config.NUM_LABELS)
    print(f"len(tokenizer_vocab) = {len(tok_vocab)}")
    model.resize_token_embeddings(len(tok_vocab))    
    trainer = Trainer(
        model=model,                         # the instantiated Transformers model to be trained
        args=training_args,                  # training arguments, defined above
        train_dataset=ds_train,              # training dataset
        eval_dataset=ds_val,                 # evaluation dataset
        compute_metrics=compute_metrics,     # the callback that computes metrics of interest
        data_collator=data_collator,
        tokenizer=tokenizer
    )
    trainer.train()
    trainer.save_model(Config.OUT_DIR + fold_str)
    df_val_fold = get_oof_preds(trainer, ds_val, df_val_fold) 
    # display(df_val_fold.head())
    df_val_preds = pd.concat([df_val_preds, df_val_fold], axis=0)
    # export the oof predictions to csv for later use in stacking
    if Config.RUNTIME != "KAGGLE":
        df_val_fold.to_csv(Config.VAL_PREDS_PATH + f"df_train_oof_preds_{Config.MODEL_NAME}_{fold_str}.csv")
    else:
        df_val_preds.to_csv("/kaggle/working/df_train_oof_preds.csv")
    print(f"Saved OOF predictions for fold {fold}")    
    del model, trainer
    gc.collect()
    torch.cuda.empty_cache()
    utils.delete_checkpoints(Config.OUT_DIR + fold_str)
    print(f"deleted checkpoints as best model for {fold_str} saved already")
    # Empty the trash to clear gdrive disk space
    colab_utils.empty_gdrive_trash()
    if not Config.RUN_ALL_FOLDS:
        break

if Config.RUN_ALL_FOLDS:
    df_val_preds.to_csv(Config.VAL_PREDS_PATH + f"df_train_oof_preds_{Config.MODEL_NAME}.csv")

Parameter 'function'=<function Dataset.map.<locals>.decorate.<locals>.decorated at 0x7ff48c7103b0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Running training for fold0


  0%|          | 0/30 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

Some weights of the model checkpoint at microsoft/deberta-v3-small were not used when initializing DebertaV2ForSeqClfMeanPooling: ['mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.bias', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.dense.bias', 'mask_predictions.dense.weight']
- This IS expected if you are initializing DebertaV2ForSeqClfMeanPooling from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSeqClfMeanPooling from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequen

len(tokenizer_vocab) = 128009


Epoch,Training Loss,Validation Loss,Pearson
0,No log,0.045101,0.688217
1,No log,0.031042,0.754331


Saved OOF predictions for fold 0
The below files were cleared from trash
['rng_state.pth', 'trainer_state.json', 'scaler.pt', 'scheduler.pt', 'optimizer.pt', 'training_args.bin', 'tokenizer.json', 'spm.model', 'added_tokens.json', 'special_tokens_map.json', 'tokenizer_config.json', 'pytorch_model.bin', 'config.json', 'checkpoint-229']


In [29]:
# Calculate the CV score
predictions = df_val_preds['val_preds'].values
labels = df_val_preds['score'].values
eval_preds = predictions, labels
cv_metric_dict = compute_metrics(eval_preds)
print(f"CV score = {cv_metric_dict}")

CV score = {'pearson': 0.7543307773172057}
